In [6]:
from dash import Dash, html, dcc, dash_table
import dash_leaflet as dl
from dash.dependencies import Input, Output
from pyproj import CRS
from owslib.ogcapi.features import Features
import json
import pandas as pd
from osgeo import ogr, osr
from datetime import datetime

# Initialize the Dash app
app = Dash(__name__)

# Define the OpenStreetMap basemap layer
osm_layer = dl.BaseLayer(
    dl.TileLayer(
        url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
        maxZoom=19,
        attribution="© OpenStreetMap"
    ),
    name="OpenStreetMap",
    checked=True
)

# Define the Esri imagery basemap layer
esri_layer = dl.BaseLayer(
    dl.TileLayer(
        url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        maxZoom=19,
        attribution="© Esri"
    ),
    name="Esri Imagery",
    checked=False
)

# Define the Climate Stations overlay layer
climate_stations_layer = dl.Overlay(
    dl.WMSTileLayer(
        url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&amp;version=1.3.0&amp;request=GetMap",
        layers="CLIMATE.STATIONS",
        format="image/png",
        transparent=True,
        attribution="Climate Stations",
        opacity=0.7  # Optional: Set opacity for the WMS layer
    ),
    name="Climate Stations",
    checked=True
)

# Create the map with the defined layers
vancouver_map = dl.Map(
    id="map-vancouver",
    style={'width': '100%', 'height': '50vh'},
    center=[49.2827, -123.1207],  # Coordinates for Vancouver
    zoom=10,
    children=[
        dl.LayersControl(
            [
                osm_layer,
                esri_layer,
                climate_stations_layer
            ],
            id="lc"
        )
    ]
)

# Define the Calgary map
calgary_map = dl.Map(
    id="map-calgary",
    style={'width': '100%', 'height': '100vh'},
    center=[51.0447, -114.0719],  # Coordinates for Calgary
    zoom=10,
    children=[
        dl.TileLayer(
            url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
            maxZoom=19,
            attribution="© OpenStreetMap"
        ),
        dl.WMSTileLayer(
            url="https://geo.weather.gc.ca/geomet-climate/?service=WMS&amp;version=1.3.0&amp;request=GetMap",
            layers="CLIMATE.STATIONS",
            format="image/png",
            transparent=True,
            attribution="Climate Stations",
            opacity=0.7  # Optional: Set opacity for the WMS layer
        )
    ]
)

# Define the app layout
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Vancouver', children=[
            vancouver_map,
            html.Div(id='stations-table')  # Placeholder for the table
        ]),
        dcc.Tab(label='Calgary', children=[calgary_map])
    ])
])

def get_epsg_code(lat, lon):
    crs_wgs84 = CRS("EPSG:4326")
    utm_zone = int((lon + 180) // 6) + 1
    hemisphere = 'north' if lat >= 0 else 'south'
    crs_utm = CRS.from_proj4(f"+proj=utm +zone={utm_zone} +datum=NAD83 +units=m +no_defs")
    epsg_code = crs_utm.to_epsg()
    return epsg_code

def create_markers(df):
    markers = []
    for _, row in df.iterrows():
        marker = dl.Marker(
            position=[row['LATITUDE_DD'], row['LONGITUDE_DD']],
            children=[
                dl.Popup([
                    html.B(f"Station ID: {row['STN_ID']}"),
                    html.Br(),
                    html.B(f"Station Name: {row['STATION_NAME']}"),
                    html.Br(),
                    html.B(f"First Date: {row['FIRST_DATE']}"),
                    html.Br(),
                    html.B(f"Last Date: {row['LAST_DATE']}"),
                    html.Br(),
                    html.B(f"Elevation (m): {row['ELEVATION']}"),
                    html.Br(),
                    html.B(f"Has Hourly Data?: {row['HAS_HOURLY_DATA']}")
                ])
            ]
        )
        markers.append(marker)
    return markers

@app.callback(
    [Output('map-vancouver', 'children'),
     Output('stations-table', 'children')],
    Input('map-vancouver', 'bounds')
)
def update_map_and_table(bounds):
    if bounds is None:
        return [dl.LayersControl([osm_layer, esri_layer, climate_stations_layer], id="lc"), None]

    south_west = bounds[0]
    north_east = bounds[1]
    bbox = [south_west[1], south_west[0], north_east[1], north_east[0]]

    # Retrieve stations data
    oafeat = Features("https://api.weather.gc.ca/")
    station_data = oafeat.collection_items("climate-stations", bbox=bbox)

    if "features" in station_data:
        # Create a dataframe using the information retrieved from the climate stations
        driver = ogr.GetDriverByName("GeoJSON")
        data_source = driver.Open(json.dumps(station_data), 0)
        layer = data_source.GetLayer()

        # Identification of the input spatial reference system (SRS)
        SRS_input = layer.GetSpatialRef()
        SR = osr.SpatialReference(str(SRS_input))
        epsg = SR.GetAuthorityCode(None)
        SRS_input.ImportFromEPSG(int(epsg))

        # Definition of the SRS used to project data
        SRS_projected = osr.SpatialReference()
        SRS_projected.ImportFromEPSG(4326)  # Using EPSG:4326 for WGS84

        # Transformation from input SRS to the preferred projection
        transform = osr.CoordinateTransformation(SRS_input, SRS_projected)

        # Create an empty list to store station data as dictionaries
        station_data_list = []

        for feature in layer:
            geom = feature.GetGeometryRef().Clone()
            geom.Transform(transform)
            
            # Get all feature field names
            field_names = [feature.GetFieldDefnRef(i).GetName() for i in range(feature.GetFieldCount())]

            # Get feature field values
            field_values = [feature.GetField(name) for name in field_names]

            # Create a dictionary representing the station data
            station_data_dict = dict(zip(field_names, field_values))

            # Append the dictionary to the list
            station_data_list.append(station_data_dict)

        # Create a Pandas DataFrame from the list of dictionaries
        stations_df = pd.DataFrame(station_data_list)

        # Function to convert dataframe lat and long from DMS to DD
        def dms_to_dd(dms):
            # Determine if the coordinate is negative
            sign = -1 if dms < 0 else 1

            # Make coordinate positive for calculation
            coordinate_pos = abs(dms)

            # Extract degrees, minutes, seconds
            degrees = coordinate_pos // 10000000
            minutes = (coordinate_pos // 100000) % 100
            seconds = (coordinate_pos % 100000) // 1000

            # Convert to decimal degrees
            decimal_degrees = sign * (degrees + (minutes / 60) + (seconds / 3600))

            return decimal_degrees

        # Create new columns for the converted values
        stations_df['LATITUDE_DD'] = stations_df['LATITUDE'].apply(dms_to_dd)
        stations_df['LONGITUDE_DD'] = stations_df['LONGITUDE'].apply(dms_to_dd)

        # Create markers
        markers = create_markers(stations_df)

        # Create a Dash DataTable from the DataFrame
        table = dash_table.DataTable(
            columns=[{"name": 'Station ID', "id": 'STN_ID', "type" : 'numeric'},
                     {"name": 'Station Name', "id": 'STATION_NAME', "type" : 'text'},
                     {"name": 'First Date', "id": 'FIRST_DATE', "type" : 'date'},
                     {"name": 'Last Date', "id": 'LAST_DATE', "type" : 'date'},
                     {"name": 'Elevation (m)', "id": 'ELEVATION', "type" : 'numeric'},
                     {"name": 'Has Hourly Data?', "id": 'HAS_HOURLY_DATA', "type" : 'text'},
                     ],
            data=stations_df.to_dict('records'),
            page_size=10  # Number of rows per page of the table
        )

        return [
            dl.LayersControl(
                [
                    osm_layer,
                    esri_layer,
                    climate_stations_layer,
                    dl.LayerGroup(markers)
                ],
                id="lc"
            ),
            table
        ]
    else:
        return [dl.LayersControl([osm_layer, esri_layer, climate_stations_layer], id="lc"), None]

if __name__ == '__main__':
    app.run(jupyter_mode='external')  # This works in Google Colab

AttributeError: module 'numpy' has no attribute '__version__'